In [29]:
# Merge drafted_players.csv & playerResponses.csv and include name all of drafted_players columns and answers
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import statistics

answers = pd.read_csv("playerResponses.csv")
stats = pd.read_csv("drafted_players.csv")
answers.rename(columns={'playerName': 'Player'}, inplace=True)
data = pd.merge(stats, answers, on='Player', how='left', sort=False)
data.dropna(subset=['Answers'], inplace=True)
data = data[~data['Answers'].str.contains("Error fetching", na=False)]
data['Answers'] = data['Answers'].fillna(' ')
data = data.groupby('Player').agg({'Answers': ' '.join, **{col: 'first' for col in data.columns if col not in ['Player', 'Answers']}}).reset_index()
data = data[data['Answers'].apply(lambda x: len(''.join(x.split())) >= 50)]
data.sort_values(by=['Round', 'Num.'], ascending=[True, True], inplace=True)

data.head(7)

,Player,Answers,Round,Num.,Drafted By,Pos,Drafted From,GP,G,A,Pts,PIM,Last Season
6,Auston Matthews,in the sort of progress do you feel was made t...,1.0,1.0,Toronto,C,Zurich SC [Swiss-A],482.0,302.0,243.0,545.0,94.0,2023-24
57,Patrik Laine,i don't know what you expected out of this sea...,1.0,2.0,Winnipeg,L,Tappara Tampere [SM-liiga],463.0,199.0,181.0,380.0,179.0,2023-24
34,Jesse Puljujarvi,uh yese welcome back I just wonder how your uh...,1.0,4.0,Edmonton,R,Karpat [SM-liiga],334.0,51.0,63.0,114.0,108.0,2022-23
54,Olli Juolevi,oh yeah of course I'm specific and you're play...,1.0,5.0,Vancouver,D,London Knights [OHL],41.0,2.0,1.0,3.0,6.0,2021-22
4,Alexander Nylander,power-play goal I darling look like you put it...,1.0,8.0,Buffalo,L,Mississauga Steelheads [OHL],93.0,14.0,20.0,34.0,20.0,2022-23
70,Tyson Jost,the decision to stay and play in the BC Hockey...,1.0,10.0,Colorado,C,Penticton Vees [BCHL],413.0,54.0,80.0,134.0,154.0,2022-23
49,Michael McLeod,mikee you're really filling out I guess that's...,1.0,12.0,New Jersey,C,Mississauga Steelheads [OHL],244.0,19.0,47.0,66.0,154.0,2023-24


In [43]:
#Add new column with confidence score for each player

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_confidence_scores(response):
    # Analyze the sentiment of the response
    score = sia.polarity_scores(response)
    # The compound score is a general sentiment score, which can serve as the "confidence" score
    return score['compound']

# Add the confidence scores to the DataFrame
data['Confidence'] = data['Answers'].apply(get_confidence_scores)
data['Confidence'].head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jesse\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


6     0.9993
57    0.9999
34    0.9998
54    0.9998
4     0.9996
Name: Confidence, dtype: float64

In [44]:
correlation_matrix = data.corr()
confidence_correlation = correlation_matrix['Confidence']
print(confidence_correlation)


Round        -0.198977
Num.         -0.177872
GP            0.097079
G             0.053746
A             0.058632
Pts           0.058979
PIM           0.110765
Confidence    1.000000
Name: Confidence, dtype: float64


In [50]:
filtered_data = data[data['Round'] > 1]
correlation_matrix = filtered_data.corr()
confidence_correlation = correlation_matrix['Confidence']
print(confidence_correlation)

Round         0.247170
Num.          0.334892
GP           -0.183112
G            -0.439753
A            -0.280874
Pts          -0.393203
PIM           0.318957
Confidence    1.000000
Name: Confidence, dtype: float64
